In [2]:
import math

In [16]:
data = open('data/day18.txt', 'r').read()  # All 'S' need to be replaced with 'SS'

In [4]:
example = """R 6 (#70c710)
D 5 (#0dc571)
L 2 (#5713f0)
D 2 (#d2c081)
R 2 (#59c680)
D 2 (#411b91)
L 5 (#8ceee2)
U 2 (#caa173)
L 1 (#1b58a2)
U 2 (#caa171)
R 2 (#7807d2)
U 3 (#a77fa3)
L 2 (#015232)
U 2 (#7a21e3)"""

In [59]:
def create_grid(data):
    movements = {'U': (-1, 0), 'R': (0, 1), 'D': (1, 0), 'L': (0, -1)}
    coords = [(0, 0)]
    curr_coord = (0, 0)
    
    # Get all grid coordinates
    for line in data.split('\n'):
        direction, steps, color = line.split(' ')
        row_movement, col_movement = movements[direction]
        for i in range(int(steps)):
            curr_row, curr_col = curr_coord
            new_coord = (curr_row + row_movement, curr_col + col_movement)
            coords.append(new_coord)
            curr_coord = new_coord
            
    # Adjust all grid coordinates
    border = set(coords)
    rows = [row for (row, col) in border]
    cols = [col for (row, col) in border]
    min_row, min_col = min(rows), min(cols)
    row_adder = abs(min_row) if min_row < 0 else 0
    col_adder = abs(min_col) if min_col < 0 else 0
    adjusted_border = [(row + row_adder, col + col_adder) for (row, col) in border]      
            
    # Create grid
    row_bound = max([row for (row, col) in adjusted_border]) + 1
    col_bound = max([col for (row, col) in adjusted_border]) + 1
    grid = [['#' if (row, col) in adjusted_border else '.' for col in range(col_bound)] for row in range(row_bound)]
       
    return grid, adjusted_border

In [60]:
def is_interior_tile(grid, row, col):
    """
    Validate whether the given tile is an interior tile    
    If it crosses an odd # of boundary lines, it's an inside tile
    If it crosses an even # of boundary lines, it's an outside tile
    """
    lines_crossed = 0
    curr_col = col
    
    # Count the # of times the point crosses a vertical line going to the left boundary
    while curr_col >= 0:
        if grid[row][curr_col] in ['J', '|', 'L']:
            lines_crossed += 1
        curr_col -= 1
        
    return (lines_crossed % 2 != 0)

In [61]:
def define_interior(grid, border):
    movements = {'U': (-1, 0), 'R': (0, 1), 'D': (1, 0), 'L': (0, -1)}
    pipes = {'D|U': '|', 'L|R':  '-', 'R|U': 'L', 'L|U': 'J', 'D|L': '7', 'D|R': 'F'}

    # Swap out # for their appropriate pipes
    new_grid = [['.' for col in range(len(grid[row]))] for row in range(len(grid))]
    for (row, col) in border:
        # Find neighbors
        potential_neighbors = [(direction, (row + row_movement, col + col_movement)) 
                               for (direction, (row_movement, col_movement)) in movements.items()]
        neighbor_directions = [direction for (direction, (row, col)) in potential_neighbors if (row, col) in border]
        # Define pipe type
        new_grid[row][col] = pipes['|'.join(sorted(neighbor_directions))]
    
    # Define interior coordinates
    interior_coords = []
    for row in range(len(new_grid)):
        for col in range(len(new_grid[row])):
            if new_grid[row][col] == '.':
                if is_interior_tile(new_grid, row, col):
                    interior_coords.append((row, col))
                    new_grid[row][col] = '#'
                    
    return new_grid, interior_coords
        

In [62]:
def part1(data):
    grid, border = create_grid(data)
    new_grid, interior_coords = define_interior(grid, border)
    
    return len(border) + len(interior_coords)

In [63]:
part1(data)

58550

In [258]:
def get_corners(data):
    directions = {'0': 'R', '1': 'D', '2': 'L', '3': 'U'}
    
    coords = [(0, 0)]
    curr_coord = (0, 0)
    
    # Get all grid coordinates
    corners = []
    num_boundary_points = 0
    for line in data.split('\n'):
        hex_code = line.split(' ')[-1]
        steps = int(hex_code[2:-2], 16)
        direction = directions[hex_code[-2]]
        curr_row, curr_col = curr_coord
        if direction == 'R':
            curr_coord = curr_row, curr_col + steps
        elif direction == 'L':
            curr_coord = curr_row, curr_col - steps
        elif direction == 'U':
            curr_coord = curr_row - steps, curr_col
        else:
            curr_coord = curr_row + steps, curr_col
        corners.append(curr_coord)
        num_boundary_points += steps

    return corners, num_boundary_points

In [263]:
def shoelace_theorem(corners):
    """
    Calculates the area of a polygon given it's vertices
    """
    rows = [row for (row, col) in corners]
    cols = [col for (row, col) in corners]
    
    return 0.5 * abs(sum(np.array(rows, dtype='int64') * np.array(cols[1:] + cols[:1], dtype='int64')) - sum(np.array(cols, dtype='int64') * np.array(rows[1:] + rows[:1], dtype='int64')))

In [264]:
def find_num_interior_points(area, num_boundary_points):
    """
    Utilizes Pick's Theorem - Area = Interior Points + 1/2 Boundary Points - 1
    """
    return area + 1 - 0.5 * num_boundary_points

In [265]:
def part2(data):
    corners, num_boundary_points = get_corners(data)
    area = shoelace_theorem(corners)
    num_interior_points = find_num_interior_points(area, num_boundary_points)
    total_points = num_boundary_points + num_interior_points
    
    return total_points

In [267]:
part2(data)

47452118468566.0